In [2]:
import pandas as pd
import numpy as np
import glob
import nibabel as nib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
def grab_stat_files(dir:str, threshold:str):
    # Set directory containing the contrast files
    dir = dir
    # Get list of all .nii.gz files in the directory
    files = glob.glob(f'{dir}/*{threshold}.nii.gz')
    # Exclude files that contain 'bin' in their names
    files = [f for f in files if 'bin' not in f]
    # Separate files into HC and SCZ groups based on filename prefixes
    HC = []
    SCZ = []
    for f in files: 
        if f.split('/')[-1].startswith('0'):
            HC.append(f)
        if f.split('/')[-1].startswith('5'):
            SCZ.append(f)
    # Sort the lists to maintain consistent order
    return sorted(HC), sorted(SCZ)

In [4]:
c = 'contrast_2v1'
thr = '2.81'
thr_hc, thr_scz = grab_stat_files(f'/BICNAS2/tuominen/ANM_SCZ/anm_files/{c}', thr)

In [5]:
def extract_pca_values(file_list):
    pca_values = []
    for file in file_list:
        img = nib.load(file)
        data = img.get_fdata()
        flattened_data = data.flatten()
        #print(flattened_data.shape)
        pca_values.append(flattened_data)
        X = np.vstack(pca_values)
    return X

In [6]:
pca_array = extract_pca_values(thr_hc + thr_scz)

In [7]:
standard_pca_array = StandardScaler().fit_transform(pca_array)

In [8]:
pca = PCA(n_components=0.95)

In [9]:
X_pca = pca.fit_transform(standard_pca_array)

In [11]:
pca.n_components_

38

In [15]:
X_pca.shape

(59, 38)